In [ ]:
!pip install tensorflow -U 

In [ ]:
from models import getModel 
from dataloader import data_pipeline #, data_pipeline_with_augmentation
from tensorflow.keras.applications.efficientnet import EfficientNetB5
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B3
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.applications.inception_resnet_v2 import InceptionResNetV2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.applications.regnet import RegNetX032 
from tensorflow.keras.applications.xception import Xception 
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau, CSVLogger, EarlyStopping 
import pandas as pd
import numpy as np
import pickle, os, shutil
from IPython.display import clear_output 

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

In [ ]:
import subprocess

datasets = [
    'classification(dilated-masks)',
    'classification(dilated-masks)(withGAN)',
    'classification(masks)',
    'classification(withGAN)',
    'classification(no-mask)',
]

In [ ]:
import subprocess

for dataset in datasets:
    subprocess.Popen(f"unzip 'drive/MyDrive/{dataset}.zip'", shell=True) # run only once

In [ ]:
preTrained = {
   'Xception' : dict(
      modelFunction = Xception,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'VGG16' : dict(
      modelFunction = VGG16,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'RegNetX032' : dict(
      modelFunction = RegNetX032,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'MobileNetV2' : dict(
      modelFunction = MobileNetV2,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'ResNet50V2' : dict(
      modelFunction = ResNet50V2,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'EfficientNetV2B3' : dict(
      modelFunction = EfficientNetV2B3,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'InceptionResNetV2' : dict(
      modelFunction = InceptionResNetV2,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
   'EfficientNetB5' : dict(
      modelFunction = EfficientNetB5,
      learningRate = 5e-6, 
      batchSize = 16,
      dropout = [0.7, 0.5, 0.4],
      trainbatchnorm = False,
      denseSize = [4096, 4096, 4096]
   ),
}

# preTrained = [
#    ('Xception', Xception, 1e-4, 16, 0.7),
#    ('VGG16', VGG16, 5e-5, 8, 0.4),
#    ('RegNetX032', RegNetX032, 5e-5, 8),
#    ('MobileNetV2', MobileNetV2, 5e-5, 8),
#    ('ResNet50V2', ResNet50V2, 5e-5, 8),
#    ('EfficientNetV2B3', EfficientNetV2B3, 5e-6, 8),
#    ('InceptionResNetV2', InceptionResNetV2, 5e-6, 8),
#    ('EfficientNetB5', EfficientNetB5, 5e-6, 8),
# ]


In [ ]:
!rm -rf results 
os.mkdir('results') 
for dataset in datasets:
  os.mkdir(f'results/{dataset}') 

In [ ]:
def train():
  for preTrainedModel in preTrained.keys():
      for dataset in datasets:
        modelFunction, learningRate, batchSize, dropout, trainbatchnorm, denseSize = preTrained[preTrainedModel].values()
        epochs = 15
        trainDataset, validateDataset = data_pipeline(DIR=dataset, BATCHSIZE=batchSize, IMAGESIZE=(256, 256)) 
        model = getModel(modelFunction, learning_rate=learningRate, trainbatchnorm=trainbatchnorm, dropout=dropout, denseSize=denseSize)

        callbacks = [ 
            EarlyStopping(patience=10, verbose=1), 
            ReduceLROnPlateau(factor=0.5, cooldown=0, patience=3, min_lr=0.5e-6), 
            CSVLogger(f'results/{dataset}/{preTrainedModel}.csv'), 
            ModelCheckpoint(f'results/{dataset}/{preTrainedModel}.hdf5', monitor = 'val_Accuracy', verbose = 1, mode='max', save_best_only=True) 
        ]

        model.fit(trainDataset, batch_size=batchSize, epochs=epochs, verbose=True, validation_data=validateDataset, shuffle=True, callbacks=callbacks) 
        model.load_weights(f'results/{dataset}/{preTrainedModel}.hdf5')

        results = model.get_metrics_result()
        results = {metric:value.numpy() for metric, value in results.items()}
        results['F1_Score'] = (2*results['recall']*results['precision'])/(results['recall']+results['precision']) 
        results['Dataset'] = dataset
        results['learningRate'] = learningRate
        results['batchSize'] = batchSize
        results['epochs'] = epochs
        df = pd.DataFrame([results])

        acc = np.round(results['Accuracy'], decimals=2)
        pickle.dump(df.to_latex(), open("results/%s/%s_report_acc=%.2f.txt"%(dataset, preTrainedModel, acc), 'wb')) 
        clear_output(wait=True) 

train() 

In [ ]:
shutil.copytree("/content/results","/content/drive/MyDrive/results", dirs_exist_ok=True) 